In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../data/processed/Totalchargesfixed.csv')
df.head(5)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:

print("\n1. HIGH-RISK CUSTOMER PROFILES")
print("-" * 30)

risk_factors = {}

contract_risk = df.groupby('Contract')['Churn'].apply(lambda x: (x == 'Yes').mean() * 100)
risk_factors['Contract'] = contract_risk.to_dict()


payment_risk = df.groupby('PaymentMethod')['Churn'].apply(lambda x: (x == 'Yes').mean() * 100)
risk_factors['PaymentMethod'] = payment_risk.to_dict()

internet_risk = df.groupby('InternetService')['Churn'].apply(lambda x: (x == 'Yes').mean() * 100)
risk_factors['InternetService'] = internet_risk.to_dict()


print("HIGHEST RISK CUSTOMER PROFILE:")
highest_risks = []
for category, risks in risk_factors.items():
    highest = max(risks.items(), key=lambda x: x[1])
    highest_risks.append(f"{category}: {highest[0]} ({highest[1]:.1f}% churn)")
    print(f"• {category}: {highest[0]} ({highest[1]:.1f}% churn)")

# Tenure analysis
short_tenure_churn = (df[df['tenure'] <= 12]['Churn'] == 'Yes').mean() * 100
print(f"• Tenure: New customers (≤12 months) ({short_tenure_churn:.1f}% churn)")


1. HIGH-RISK CUSTOMER PROFILES
------------------------------
HIGHEST RISK CUSTOMER PROFILE:
• Contract: Month-to-month (42.7% churn)
• PaymentMethod: Electronic check (45.3% churn)
• InternetService: Fiber optic (41.9% churn)
• Tenure: New customers (≤12 months) (47.4% churn)


In [4]:
print("\n2. RETENTION OPPORTUNITIES")
print("-" * 30)

service_features = ['OnlineSecurity', 'OnlineBackup', 'TechSupport', 'DeviceProtection']
protective_services = {}

for service in service_features:
    if service in df.columns:
        with_service = (df[df[service] == 'Yes']['Churn'] == 'Yes').mean() * 100
        without_service = (df[df[service] == 'No']['Churn'] == 'Yes').mean() * 100
        
        if with_service < without_service:
            reduction = without_service - with_service
            protective_services[service] = reduction

print("SERVICES THAT REDUCE CHURN:")
for service, reduction in sorted(protective_services.items(), key=lambda x: x[1], reverse=True):
    print(f"• {service}: Reduces churn by {reduction:.1f} percentage points")

long_contract_churn = (df[df['Contract'].isin(['One year', 'Two year'])]['Churn'] == 'Yes').mean() * 100
month_to_month_churn = (df[df['Contract'] == 'Month-to-month']['Churn'] == 'Yes').mean() * 100
contract_benefit = month_to_month_churn - long_contract_churn

print(f"• Long-term Contracts: Reduce churn by {contract_benefit:.1f} percentage points")



2. RETENTION OPPORTUNITIES
------------------------------
SERVICES THAT REDUCE CHURN:
• OnlineSecurity: Reduces churn by 27.2 percentage points
• TechSupport: Reduces churn by 26.5 percentage points
• OnlineBackup: Reduces churn by 18.4 percentage points
• DeviceProtection: Reduces churn by 16.6 percentage points
• Long-term Contracts: Reduce churn by 36.0 percentage points


In [6]:

print("\n3. REVENUE IMPACT ANALYSIS")
print("-" * 30)


total_customers = len(df)
churned_customers = len(df[df['Churn'] == 'Yes'])
avg_monthly_revenue = df['MonthlyCharges'].mean()
avg_churned_revenue = df[df['Churn'] == 'Yes']['MonthlyCharges'].mean()


annual_revenue_loss = churned_customers * avg_churned_revenue * 12
total_annual_revenue = total_customers * avg_monthly_revenue * 12
revenue_loss_percentage = (annual_revenue_loss / total_annual_revenue) * 100

print(f"INANCIAL IMPACT:")
print(f"• Total Customers: {total_customers:,}")
print(f"• Churned Customers: {churned_customers:,}")
print(f"• Average Monthly Revenue per Customer: ${avg_monthly_revenue:.2f}")
print(f"• Annual Revenue Loss: ${annual_revenue_loss:,.0f}")
print(f"• Revenue Loss Percentage: {revenue_loss_percentage:.1f}%")


high_value_customers = df[df['MonthlyCharges'] > df['MonthlyCharges'].quantile(0.75)]
high_value_churn_rate = (high_value_customers['Churn'] == 'Yes').mean() * 100
high_value_loss = len(high_value_customers[high_value_customers['Churn'] == 'Yes']) * high_value_customers['MonthlyCharges'].mean() * 12

print(f"• High-Value Customer Churn Rate: {high_value_churn_rate:.1f}%")
print(f"• High-Value Customer Revenue Loss: ${high_value_loss:,.0f}")


3. REVENUE IMPACT ANALYSIS
------------------------------
INANCIAL IMPACT:
• Total Customers: 7,043
• Churned Customers: 1,869
• Average Monthly Revenue per Customer: $64.76
• Annual Revenue Loss: $1,669,570
• Revenue Loss Percentage: 30.5%
• High-Value Customer Churn Rate: 32.9%
• High-Value Customer Revenue Loss: $699,835


In [7]:

print("\n4. ACTIONABLE RECOMMENDATIONS")
print("-" * 30)

print("IMMEDIATE ACTIONS:")
print("1. TARGET HIGH-RISK CUSTOMERS:")
print("   • Month-to-month contract customers")
print("   • Electronic check payment users")
print("   • Customers with <12 months tenure")

print("\n2. RETENTION STRATEGIES:")
print("   • Promote long-term contracts with incentives")
print("   • Encourage automatic payment methods")
print("   • Offer security services to fiber optic customers")

print("\n3. SERVICE IMPROVEMENTS:")
if protective_services:
    top_service = max(protective_services.keys(), key=lambda x: protective_services[x])
    print(f"   • Promote {top_service} (highest retention impact)")
print("   • Focus on customer onboarding (first 12 months)")
print("   • Improve fiber optic service experience")

print("\n4. REVENUE PROTECTION:")
print(f"   • Potential annual savings: ${annual_revenue_loss * 0.1:,.0f}")
print("     (10% reduction in churn)")
print(f"   • Focus on high-value customers (${high_value_loss:,.0f} at risk)")


4. ACTIONABLE RECOMMENDATIONS
------------------------------
IMMEDIATE ACTIONS:
1. TARGET HIGH-RISK CUSTOMERS:
   • Month-to-month contract customers
   • Electronic check payment users
   • Customers with <12 months tenure

2. RETENTION STRATEGIES:
   • Promote long-term contracts with incentives
   • Encourage automatic payment methods
   • Offer security services to fiber optic customers

3. SERVICE IMPROVEMENTS:
   • Promote OnlineSecurity (highest retention impact)
   • Focus on customer onboarding (first 12 months)
   • Improve fiber optic service experience

4. REVENUE PROTECTION:
   • Potential annual savings: $166,957
     (10% reduction in churn)
   • Focus on high-value customers ($699,835 at risk)


In [8]:

print("\n5. KEY METRICS TO MONITOR")
print("-" * 30)

print("TRACK THESE METRICS:")
print("• Monthly churn rate by customer segment")
print("• Contract conversion rate (month-to-month to long-term)")
print("• Service adoption rate for new customers")
print("• Early warning indicators (payment delays, service calls)")
print("• Revenue per customer trend")

# Summary dashboard data
print("\n6. EXECUTIVE SUMMARY")
print("-" * 30)

churn_rate = (df['Churn'] == 'Yes').mean() * 100
print(f"KEY NUMBERS:")
print(f"• Overall Churn Rate: {churn_rate:.1f}%")
print(f"• Highest Risk Segment: Month-to-month contracts ({contract_risk['Month-to-month']:.1f}% churn)")
print(f"• Annual Revenue at Risk: ${annual_revenue_loss:,.0f}")
print(f"• Potential Savings (10% churn reduction): ${annual_revenue_loss * 0.1:,.0f}")

if protective_services:
    best_service = max(protective_services.keys(), key=lambda x: protective_services[x])
    print(f"• Best Retention Tool: {best_service}")




5. KEY METRICS TO MONITOR
------------------------------
TRACK THESE METRICS:
• Monthly churn rate by customer segment
• Contract conversion rate (month-to-month to long-term)
• Service adoption rate for new customers
• Early warning indicators (payment delays, service calls)
• Revenue per customer trend

6. EXECUTIVE SUMMARY
------------------------------
KEY NUMBERS:
• Overall Churn Rate: 26.5%
• Highest Risk Segment: Month-to-month contracts (42.7% churn)
• Annual Revenue at Risk: $1,669,570
• Potential Savings (10% churn reduction): $166,957
• Best Retention Tool: OnlineSecurity
